In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import optimize

In [2]:
dataframe = pd.read_csv("./FATS_GAIA.dat")
dataframe.head()

,Amplitude,AndersonDarling,Autocor_length,Class,Con,Eta_e,FluxPercentileRatioMid20,FluxPercentileRatioMid35,FluxPercentileRatioMid50,FluxPercentileRatioMid65,...,PeriodLS,Period_fit,Psi_CS,Psi_eta,Q31,Rcs,Skew,SlottedA_length,SmallKurtosis,Std
0,1.416432,0.949482,3.0,MIRA_SR,0.000000,5.442293,0.243054,0.462373,0.653511,0.761790,...,268.331543,0.025528,0.304804,0.310355,1.784879,0.304804,0.011895,19.899170,-1.266215,0.931676
1,0.443386,1.000000,1.0,RRAB,0.000000,12042.752633,0.162446,0.231268,0.517775,0.720122,...,0.508910,0.012390,0.309458,0.519645,0.397205,0.258203,-0.962335,0.147949,-0.183573,0.284637
2,0.170099,0.875986,4.0,MIRA_SR,0.030303,137.020266,0.033964,0.208724,0.378623,0.501416,...,8.742769,0.040022,0.342646,0.700200,0.119849,0.368936,0.472161,0.147949,0.064528,0.089020
3,1.350858,0.999869,4.0,MIRA_SR,0.000000,23.583559,0.259476,0.593270,0.613808,0.817245,...,379.949707,0.000843,0.349687,0.285682,1.867060,0.349687,-0.047072,23.597900,-1.454570,0.926506
4,0.248472,0.999947,3.0,MIRA_SR,0.000000,125.486491,0.202240,0.448444,0.565120,0.606539,...,318.427795,0.022169,0.247451,0.578087,0.273482,0.247451,0.594605,0.147949,-0.689124,0.153349


## Preprocesamiento: dividir base de datos en *train* y *test*

In [3]:
train = dataframe.sample(frac=0.8, random_state=7).reset_index()
train = train.drop("index", 1)
test = dataframe.drop(train.index).reset_index()
test = test.drop("index", 1)

In [4]:
X_train = train[[column for column in train.columns if column != "PeriodLS"]]
y_train = train["PeriodLS"]
X_test = test[[column for column in train.columns if column != "PeriodLS"]]
y_test = test["PeriodLS"]

In [5]:
display(X_train.head())
display(y_train.head())
display(X_test.head())
display(y_test.head())

,Amplitude,AndersonDarling,Autocor_length,Class,Con,Eta_e,FluxPercentileRatioMid20,FluxPercentileRatioMid35,FluxPercentileRatioMid50,FluxPercentileRatioMid65,...,PercentDifferenceFluxPercentile,Period_fit,Psi_CS,Psi_eta,Q31,Rcs,Skew,SlottedA_length,SmallKurtosis,Std
0,0.165094,0.711871,2.0,MIRA_SR,0.0,149.154338,0.286535,0.315403,0.354876,0.634872,...,0.022645,0.585891,0.267306,1.110589,0.106045,0.345638,0.111140,28.406250,-0.350345,0.092656
1,0.233673,0.999928,1.0,RRC,0.0,84885.514569,0.175111,0.320062,0.454982,0.524935,...,0.024830,0.999908,0.347286,1.039225,0.206278,0.233412,-0.874187,0.073975,-0.152506,0.152241
2,0.490221,0.999994,1.0,RRAB,0.0,17974.650153,0.329906,0.394319,0.508053,0.632579,...,0.059739,0.005520,0.281397,0.759796,0.447899,0.197481,-0.641018,0.073975,-0.613068,0.293194
3,0.538722,1.000000,1.0,RRAB,0.0,21720.314290,0.209576,0.400352,0.481303,0.588752,...,0.065208,0.000047,0.288827,0.319746,0.522474,0.122131,-0.722468,0.073975,-0.730431,0.353988
4,0.256679,0.951044,1.0,RRAB,0.0,14413.913756,0.235947,0.332411,0.451095,0.547342,...,0.028744,0.078749,0.325063,0.708258,0.210350,0.308932,-0.609829,0.073975,-0.205571,0.155104


0    1.224964
1    3.954775
2    0.529830
3    0.482072
4    0.867182
Name: PeriodLS, dtype: float64

,Amplitude,AndersonDarling,Autocor_length,Class,Con,Eta_e,FluxPercentileRatioMid20,FluxPercentileRatioMid35,FluxPercentileRatioMid50,FluxPercentileRatioMid65,...,PercentDifferenceFluxPercentile,Period_fit,Psi_CS,Psi_eta,Q31,Rcs,Skew,SlottedA_length,SmallKurtosis,Std
0,0.327284,0.991928,1.0,RRAB,0.0,58412.807332,0.152161,0.227720,0.426300,0.598559,...,0.033710,0.395342,0.338043,0.974978,0.303738,0.185923,-0.768070,0.073975,0.206281,0.186596
1,0.257141,0.698589,1.0,RRAB,0.0,17613.263062,0.151067,0.235604,0.477704,0.705610,...,0.025574,0.053945,0.316196,0.496841,0.250239,0.173091,-0.080403,0.147949,-0.799777,0.155838
2,0.221411,0.999640,1.0,RRC,0.0,44426.219390,0.416797,0.524806,0.699892,0.818684,...,0.022104,0.213965,0.326893,0.568963,0.293938,0.185099,-0.039193,0.073975,-1.558781,0.156909
3,0.202297,0.999475,1.0,RRC,0.0,28565.796345,0.286940,0.576348,0.650045,0.851497,...,0.020635,0.429962,0.245748,1.199581,0.283047,0.205665,-0.236193,0.073975,-1.387901,0.141242
4,0.230783,0.999916,1.0,RRC,0.0,46618.997454,0.175520,0.341385,0.539694,0.895385,...,0.026930,0.948142,0.278585,1.516252,0.281733,0.236349,-0.477540,0.073975,-1.041804,0.160290


0    0.546369
1    0.831368
2    0.829974
3    0.951642
4    0.490445
Name: PeriodLS, dtype: float64

## Construcción del árbol

In [6]:
class Node:
    id_count = 0

    def __init__(self, X, y, parent=None):
        self.X = X
        self.y = y
        self.parent = parent
        self.id = Node.id_count
        Node.id_count += 1
        self.children = dict()

    def add_child(self, child_X, child_y):
        new_node = Node(child_X, child_y, self)
        self.children.update({new_node.id: new_node})
        return new_node

    def find_split(self):
        total_node_variance = self.y.var()
        features_var_red = {}
        for feature in self.X.columns:
            print(feature)
            feature_values = self.X[feature].unique()
            values_var_red = []
            for value in feature_values:
                left_side = self.X[self.X[feature] <= value]
                right_side = self.X[self.X[feature] > value]
                left_side_var = self.y[self.y.index.isin(left_side.index)]
                right_side_var = self.y[self.y.index.isin(right_side.index)]
                final_var = total_node_variance - \
                    (left_side_var + right_side_var)
                values_var_red.append(final_var)
            max_var_red = max(values_var_red)
            features_var_red[feature] = max_var_red
        return max(features_var_red, key=features_var_red.get)

    @staticmethod
    def var_red_function(value, feature, X_values, y_values):
        left_side = X_values[X_values[feature] <= value]
        right_side = X_values[X_values[feature] > value]
        left_side_y = y_values[y_values.index.isin(left_side.index)]
        right_side_y = y_values[y_values.index.isin(right_side.index)]
        return left_side_y.var() + right_side_y.var()

    def split_with_scipy(self):
        valid_columns = [
            column for column in self.X.columns if column != "Class"
        ]
        features_var_red = dict()
        for feature in valid_columns:
            feature_values = self.X[feature].unique()
            if min(feature_values) == max(feature_values):
                continue
            optimal_split, split_var_red, ierr, numf = scipy.optimize.fminbound(
                Node.var_red_function,
                min(feature_values),
                max(feature_values),
                args=(feature, self.X, self.y),
                full_output=1)
            features_var_red[feature] = (split_var_red, optimal_split)
        best_split = sorted(features_var_red.items(), key=lambda x: x[1])[0]
        self.split_label = best_split[0]
        self.split_value = best_split[1][-1]
        return self.split_label, self.split_value

In [7]:
def fit(data, target, parent, depth, max_depth=10, min_values=5):
    print("Current depth: {}".format(depth))
    root = Node(data, target, parent)
    print("There are {} rows in current node".format(len(root.X)))
    if len(root.X) <= min_values:
        return root
    if depth >= max_depth:
        return root
    
    root.split_with_scipy()
    split_label = root.split_label
    split_value = root.split_value
    print(split_label, split_value)
    left_data = root.X[root.X[split_label] <= split_value]
    right_data = root.X[root.X[split_label] > split_value]
    left_target = root.y[root.y.index.isin(left_data.index)]
    right_target = root.y[root.y.index.isin(right_data.index)]
    
    root.left = fit(left_data, left_target, root, depth + 1, max_depth, min_values)
    root.right = fit(right_data, right_target, root, depth + 1, max_depth, min_values)
    return root

In [8]:
X_train.describe()

,Amplitude,AndersonDarling,Autocor_length,Con,Eta_e,FluxPercentileRatioMid20,FluxPercentileRatioMid35,FluxPercentileRatioMid50,FluxPercentileRatioMid65,FluxPercentileRatioMid80,...,PercentDifferenceFluxPercentile,Period_fit,Psi_CS,Psi_eta,Q31,Rcs,Skew,SlottedA_length,SmallKurtosis,Std
count,160535.000000,160535.000000,160535.000000,160535.000000,1.605350e+05,160535.000000,160535.000000,160535.000000,160535.000000,160535.000000,...,160535.000000,160535.000000,160535.000000,160535.000000,160535.000000,160535.000000,160535.000000,160535.000000,160535.000000,160535.000000
mean,0.356419,0.937261,2.062827,0.001201,2.965722e+04,0.206519,0.355330,0.512187,0.673373,0.860809,...,0.044384,0.295846,0.306734,0.692503,0.342336,0.262005,-0.188360,5.877419,-0.348269,0.221695
std,0.271572,0.142283,1.584550,0.007199,3.643872e+06,0.112071,0.142592,0.151650,0.143288,0.103616,...,0.041662,0.353279,0.059721,0.375889,0.313911,0.094452,0.603974,9.704988,1.149212,0.178293
min,0.051430,0.110900,1.000000,0.000000,4.444413e-01,0.001687,0.003167,0.006041,0.012819,0.088982,...,0.004883,0.000000,0.133990,0.010864,0.007683,0.029167,-4.447674,0.073853,-2.033755,0.025378
25%,0.186208,0.963468,1.000000,0.000000,1.784592e+02,0.124199,0.255661,0.412344,0.582547,0.807629,...,0.023019,0.008256,0.261333,0.406675,0.164870,0.184959,-0.595983,0.073975,-1.070421,0.115112
50%,0.278767,0.998832,1.000000,0.000000,8.025395e+03,0.193074,0.348707,0.513457,0.682174,0.884615,...,0.032474,0.112437,0.297783,0.623679,0.266370,0.250399,-0.230937,0.147949,-0.630801,0.170855
75%,0.422732,0.999993,3.000000,0.000000,3.105708e+04,0.271996,0.447624,0.615570,0.779028,0.939179,...,0.045371,0.533377,0.345897,0.910646,0.385715,0.339176,0.169143,11.909912,0.028813,0.257111
max,3.452071,1.000000,34.000000,0.125000,1.459391e+09,0.912962,0.969089,0.981342,0.994295,0.999801,...,0.870540,1.000000,0.496244,3.019829,3.143382,0.496244,4.415321,135.817383,26.480636,1.747553


In [9]:
fit(X_train, y_train, None, 0)

Current depth: 0
There are 160535 rows in current node
PercentAmplitude 0.6629166163751231
Current depth: 1
There are 160533 rows in current node
Rcs 0.032280178715976604
Current depth: 2
There are 3 rows in current node
Current depth: 2
There are 160530 rows in current node
Rcs 0.036241764513365846
Current depth: 3
There are 7 rows in current node
Freq2_harmonics_rel_phase_2 -0.1407084967330724
Current depth: 4
There are 2 rows in current node
Current depth: 4
There are 5 rows in current node
Current depth: 3
There are 160523 rows in current node
Rcs 0.03849862655405105
Current depth: 4
There are 3 rows in current node
Current depth: 4
There are 160520 rows in current node
Rcs 0.03943976004021325
Current depth: 5
There are 4 rows in current node
Current depth: 5
There are 160516 rows in current node
Rcs 0.04038972965637586
Current depth: 6
There are 2 rows in current node
Current depth: 6
There are 160514 rows in current node
Rcs 0.04079182785787758
Current depth: 7
There are 2 rows i

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/optimize/optimize.py:1746: RuntimeWarning: overflow encountered in long_scalars
  xm = 0.5 * (a + b)


Amplitude 0.37722089123591035
Current depth: 9
There are 1 rows in current node
Current depth: 9
There are 6 rows in current node
Eta_e 1981.165036192062
Current depth: 10
There are 3 rows in current node
Current depth: 10
There are 3 rows in current node
Current depth: 8
There are 160505 rows in current node
Rcs 0.04298542876854104
Current depth: 9
There are 3 rows in current node
Current depth: 9
There are 160502 rows in current node
Rcs 0.043759269503051555
Current depth: 10
There are 4 rows in current node
Current depth: 10
There are 160498 rows in current node
Current depth: 1
There are 2 rows in current node
